In [56]:
import numpy as np

# Classe CalculCovariance
class CalculCovariance:
    def __init__(self, p_points, stride):
        """
        Initialise une nouvelle instance de la classe CalculCovariance.
        
        :param p_points: Nombre de points de données pour chaque calcul de matrice de covariance.
        :param stride: Intervalle de décalage entre chaque calcul de matrice de covariance.
        """
        self.p_points = p_points  # Nombre de points de données pour chaque calcul de matrice de covariance
        self.stride = stride  # Intervalle de décalage entre chaque calcul de matrice de covariance
    
    def calculer_matrices_covariance(self, donnees):
        """
        Calcule les matrices de covariance pour les données fournies.
        
        :param donnees: Un numpy array où chaque ligne est une information et chaque colonne un point de données.
        :return: Liste des matrices de covariance de taille KxK, où K est le nombre de lignes dans les données.
        """
        k, n = donnees.shape  # k = nombre d'informations, n = nombre de points de données
        matrices_covariance = []
        
        for start_index in range(0, n - self.p_points + 1, self.stride):
            end_index = start_index + self.p_points
            segment = donnees[:, start_index:end_index]
            # Calculer la covariance entre les lignes (informations)
            covariance_matrix = np.cov(segment)
            matrices_covariance.append(covariance_matrix)
        
        return matrices_covariance

# Classe Trajectoire
class Trajectoire:
    def __init__(self, *covariance_matrices):
        self.covariance_matrices = list(covariance_matrices)
        self.trajectoire = self.calculer_trajectoire_initiale()
    
    def ajouter_covariance(self, nouvelle_covariance):
        self.covariance_matrices.append(nouvelle_covariance)
        self.trajectoire = self.calculer_trajectoire_apres_ajout()
    
    def calculer_trajectoire_initiale(self):
        trajectoire = np.zeros_like(self.covariance_matrices[0])
        for cov in self.covariance_matrices:
            trajectoire += cov
        return trajectoire
    
    def calculer_trajectoire_apres_ajout(self):
        trajectoire = np.zeros_like(self.covariance_matrices[0])
        for cov in self.covariance_matrices:
            trajectoire += cov
        return trajectoire
    
    def obtenir_covariances(self):
        return self.covariance_matrices
    
    def obtenir_trajectoire(self):
        return self.trajectoire
    
    def __str__(self):
        return (f'Trajectoire avec {len(self.covariance_matrices)} matrices de covariance\n'
                f'Trajectoire actuelle:\n{self.trajectoire}')

# Classe GestionnaireTrajectoires
class GestionnaireTrajectoires:
    def __init__(self, max_matrices_par_trajectoire, *initial_covariances):
        self.trajectoires = []
        self.max_matrices_par_trajectoire = max_matrices_par_trajectoire
        if initial_covariances:
            traj = Trajectoire(*initial_covariances[:max_matrices_par_trajectoire])
            self.trajectoires.append(traj)
            remaining_covariances = initial_covariances[max_matrices_par_trajectoire:]
            while remaining_covariances:
                traj = Trajectoire(*remaining_covariances[:max_matrices_par_trajectoire])
                self.trajectoires.append(traj)
                remaining_covariances = remaining_covariances[max_matrices_par_trajectoire:]
    
    def ajouter_covariance(self, nouvelle_covariance):
        if not self.trajectoires or len(self.trajectoires[-1].obtenir_covariances()) >= self.max_matrices_par_trajectoire:
            self.trajectoires.append(Trajectoire(nouvelle_covariance))
        else:
            self.trajectoires[-1].ajouter_covariance(nouvelle_covariance)
    
    def obtenir_trajectoires(self):
        return [traj.obtenir_trajectoire() for traj in self.trajectoires]
    
    def __str__(self):
        result = ''
        for i, traj in enumerate(self.trajectoires):
            result += f'Trajectoire {i+1}:\n{traj}\n'
        return result

# Fonction de test
def test_gestionnaire_trajectoires(donnees, p_points,stride, nb_cov_matrix_per_traj, file):
    file.write("----- Test GestionnaireTrajectoires -----\n")
    file.write(f"Données:\n{donnees}\n")
    file.write(f"p_points: {p_points}\n")
    file.write(f"stride: {stride}\n")
    file.write(f"Nombre max de matrices par trajectoire: {nb_cov_matrix_per_traj}\n")

    # Création de l'instance CalculCovariance
    calcul_cov = CalculCovariance(p_points,stride)
    
    # Calcul des matrices de covariance
    matrices_covariance = calcul_cov.calculer_matrices_covariance(donnees)
    
    # Instanciation du gestionnaire de trajectoires avec un maximum de p_points matrices par trajectoire
    gestionnaire = GestionnaireTrajectoires(nb_cov_matrix_per_traj, *matrices_covariance)
    
    # Écriture de l'état initial des trajectoires dans le fichier
    file.write("État initial des trajectoires:\n")
    file.write(str(gestionnaire))
    file.write("\n")
    
    # Ajout d'une nouvelle matrice de covariance
    nouvelle_cov = np.cov(donnees[:, :p_points])
    gestionnaire.ajouter_covariance(nouvelle_cov)
    
    # Écriture de l'état des trajectoires après ajout de la nouvelle matrice dans le fichier
    file.write("État des trajectoires après ajout de la nouvelle matrice:\n")
    file.write(str(gestionnaire))
    file.write("\n----- Fin du test -----\n\n")

# Exemple de tests avec différentes tailles de matrices et différents nombres de matrices par trajectoire
if __name__ == "__main__":
    # Données prédéfinies pour les tests
    donnees1 = np.array([
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        [2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
        [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    ])

    donnees2 = np.array([
        [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        [15, 25, 35, 45, 55, 65, 75, 85, 95, 105],
        [20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
        [25, 35, 45, 55, 65, 75, 85, 95, 105, 115]
    ])

    donnees3 = np.array([
        [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
        [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96],
        [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112],
        [8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128],
        [9, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99, 108, 117, 126, 135, 144]
    ])

    scenarios = [
        (donnees1, 3,1, 2),  # 3 informations, 10 points de données, 3 points par matrice, 2 matrices par trajectoire
        (donnees2, 5,1, 3),  # 4 informations, 10 points de données, 5 points par matrice, 3 matrices par trajectoire
        (donnees3, 6,1, 4),  # 5 informations, 16 points de données, 6 points par matrice, 4 matrices par trajectoire
    ]

    with open("resultats_tests.txt", "w") as file:
        for donnees, p_points,stride, nb_cov_matrix_per_traj in scenarios:
            test_gestionnaire_trajectoires(donnees, p_points,stride, nb_cov_matrix_per_traj, file)
